In [1]:
import pandas as pd
import numpy as np
import requests
import json
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials())
from pprint import pprint

In [2]:
url_1= 'http://www.acclaimedmusic.net/061024/1948-09art.htm'
artists_1= (pd.read_html(url_1,header=0))[0]["Unnamed: 1"]

pages= np.arange(2,6)
artists_list=[]

for page in pages:
    url_2= f"http://www.acclaimedmusic.net/061024/1948-09art{page}.htm"
    artists_df= (pd.read_html(url_2,header=0))
    artists_list.append(artists_df[0]["Unnamed: 1"])

artists= artists_1.append(artists_list)
artists.dropna(inplace=True)
artists_names= artists.reset_index()
artists_names.drop(artists_names.columns[0], axis=1, inplace=True)
artists_names.rename(columns={"Unnamed: 1": "Artist Name"}, inplace=True)
artists_names

,Artist Name
0,The Beatles
1,The Rolling Stones
2,Bob Dylan
3,David Bowie
4,Led Zeppelin
...,...
995,The Sundays
996,Herbert
997,Stephen Stills
998,Mazzy Star


In [3]:
artists_names["Artist ID"]= ""
artists_names.head()

,Artist Name,Artist ID
0,The Beatles,
1,The Rolling Stones,
2,Bob Dylan,
3,David Bowie,
4,Led Zeppelin,


In [4]:

for index, row in artists_names.iterrows():
    name= row['Artist Name']
    results = sp.search(q='artist:' + name, type='artist')
    
    try:
        artists_names.loc[index,'Artist ID'] = results['artists']["items"][0]["id"]
        artists_names.loc[index,'Artist_Popularity'] = results['artists']["items"][0]["popularity"]      
        artists_names.loc[index,'Artist_Followers'] = results['artists']["items"][0]["followers"]["total"]
        artists_names.loc[index,'Artist Genre']= ",".join(results['artists']["items"][0]["genres"])
        if len(results['artists']["items"][0]["genres"])==0:
            artists_names.loc[index,'Main Genre']= np.NaN 
        else:
            artists_names.loc[index,'Main Genre']= results['artists']["items"][0]["genres"][0]
                 
        
    except:
        
        #Printing an explanation message
        print(f"ID not founded for {row['Artist Name']}")
        artists_names.loc[index,'Artist ID'] = np.NaN
        artists_names.loc[index,'Artist_Popularity'] = np.NaN
        artists_names.loc[index,'Artist_Followers']= np.NaN
        artists_names.loc[index,'Artist Genre']= np.NaN
        artists_names.loc[index,'Main Genre']= np.NaN
        pass
        
# artists_names.dropna(inplace=True)
artists_names




ID not founded for Frank Zappa/The Mothers of Invention
ID not founded for Janis Joplin/Big Brother and The Holding Company
ID not founded for The Buzzcocks
ID not founded for Paul McCartney and Wings
ID not founded for The Arcade Fire
ID not founded for The Small Faces
ID not founded for The Dead Kennedys
ID not founded for Afrika Bambaataa & The Soul Sonic Force
ID not founded for Harold Melvin and The Bluenotes
ID not founded for Richard Hell & The Voidoids
ID not founded for The Mahavishnu Orchestra
ID not founded for The Flamin' Groovies
ID not founded for Question Mark and the Mysterians
ID not founded for The Nitty Gritty Dirt Band
ID not founded for The Art Ensemble of Chicago
ID not founded for Rob Base & DJ E-Z Rock
ID not founded for PM Dawn
ID not founded for Le Mystère des Voix Bulgares
ID not founded for Sam the Sham and The Pharaos
ID not founded for The Screaming Trees
ID not founded for Willie Mae "Big Mama" Thornton
ID not founded for Deutsch Amerikanische Freundschaf

,Artist Name,Artist ID,Artist_Popularity,Artist_Followers,Artist Genre,Main Genre
0,The Beatles,3WrFJ7ztbogyGnTHbHJFl2,90.0,16392273.0,"beatlesque,british invasion,classic rock,merse...",beatlesque
1,The Rolling Stones,22bE4uQ6baNwSHPVcDxLCe,84.0,8671527.0,"album rock,british invasion,classic rock,rock",album rock
2,Bob Dylan,74ASZWbe4lXaubB36ztrGX,79.0,4215667.0,"album rock,classic rock,country rock,folk,folk...",album rock
3,David Bowie,0oSGxfWSnnOXhD2fKuz2Gy,82.0,5517503.0,"album rock,art rock,classic rock,dance rock,gl...",album rock
4,Led Zeppelin,36QJpDe2go2KgaRleHCDTp,82.0,9052655.0,"album rock,classic rock,hard rock,rock",album rock
...,...,...,...,...,...,...
995,The Sundays,4nlqDmbzFzbNITaqjJv7D7,52.0,86568.0,"alternative pop,alternative rock,britpop,dream...",alternative pop
996,Herbert,5zCaQxjl110XTrm4LQ1CxY,70.0,100695.0,"classical,classical performance,orchestral per...",classical
997,Stephen Stills,4WlSvDKaq1PA2Nr7cCIPxX,56.0,163072.0,"blues rock,classic rock,country rock,folk,folk...",blues rock
998,Mazzy Star,37w38cCSGgKLdayTRjna4W,64.0,410067.0,"alternative rock,dream pop,melancholia,slow core",alternative rock


In [7]:
artists_names["Artist ID"]

Artist Name          1000
Artist ID             976
Artist_Popularity     976
Artist_Followers      976
Artist Genre          976
Main Genre            947
dtype: int64

In [5]:
top10_tracks= pd.DataFrame(columns=["Artist","Spotify_ID","Tracks"])
top10_tracks


,Artist,Spotify_ID,Tracks


In [6]:

counter=1
track_names=[]
for index, row in artists_names.iterrows():
    
    TOKEN= "BQCwZoKVR_DPg0ZfmmjlV0nv3H8ywnP2mlNoceSsDhk-2k0LoGJlhlzgP1vFxYMTX4zrm-ukoS2qGKQk_jFhlC-sZ0ZtK6MSX3br_GYesWM5MD45nv5f-5U-PBtPx50J0jS3E-Q8IIXyEzcNGw3cRzopooQNOj3c3uzym2P905DGG604jdwncjrxmXPiSWdoYSiosqDdKfLEPFG5uPqgLkONpDTIkTuZ5F87yoXchhJ-Vh9zVSdbeNCX6E9r0NKliEDTIr39yH6KoVQQwxIFsxe9P1W6zrdi"
    url = f"https://api.spotify.com/v1/artists/{row['Artist ID']}/top-tracks?country=US"
    headers = {'Authorization': "Bearer {}".format(TOKEN)}
    r = requests.get(url, headers=headers).json() 

    track = r['tracks'][:10]

    for tracks in range(len(track)):    
        try:
            top10_tracks.loc[counter,'Tracks'] = track[tracks]["name"]
            top10_tracks.loc[counter,'Artist'] = track[tracks]["album"]["artists"][0]["name"]
            top10_tracks.loc[counter,'Spotify_ID'] = track[tracks]["album"]["artists"][0]["id"]
            counter+=1
        
        except:
            #Printing an explanation message
            pass
top10_tracks    


KeyError: 'tracks'

In [20]:
grouped= top10_tracks.groupby("Artist")
count= grouped["Spotify_ID"].count()
count

Artist
!!!                                             10
$uicideBoy$                                     10
...And You Will Know Us by the Trail of Dead    10
2Pac                                            10
4th & Orange                                     2
                                                ..
case/lang/veirs                                  7
dvsn                                             1
for KING & COUNTRY                               1
k.d. lang                                        5
lovelytheband                                   10
Name: Spotify_ID, Length: 1178, dtype: int64